In [1]:
import numpy as np
import pandas as pd
import time
import datetime
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

### Подход 1: градиентный бустинг "в лоб". Задания.
Здесь буду последовательно описывать процесс выполнения заданий. Ответы на итоговые вопросы еще раз выписаны ниже (под заголовком "Подход 1. Ответы на вопросы).

1. Считываем таблицу из файла features.csv. На всякий случай выведем названия всех столбцов (сошлось с инструкцией, кто бы мог подумать!)

In [2]:
data_train = pd.read_csv('./features.csv', index_col='match_id')
data_train.head(20)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63
5,1430284186,1,11,5,1961,1461,19,0,1,6,...,4,4,0,-43.0,1453,0,512,2038,0,63
8,1430293701,1,8,3,967,1136,7,1,0,8,...,6,3,0,10.0,1968,0,1536,1983,12,63
9,1430299335,7,35,5,2117,1252,16,0,0,6,...,3,4,0,-15.0,4079,1,1540,0,63,0
11,1430308974,1,17,5,1527,906,10,0,1,7,...,3,4,0,26.0,3071,0,0,1572,0,63


In [3]:
for c in data_train.columns:
    print(c)

start_time
lobby_type
r1_hero
r1_level
r1_xp
r1_gold
r1_lh
r1_kills
r1_deaths
r1_items
r2_hero
r2_level
r2_xp
r2_gold
r2_lh
r2_kills
r2_deaths
r2_items
r3_hero
r3_level
r3_xp
r3_gold
r3_lh
r3_kills
r3_deaths
r3_items
r4_hero
r4_level
r4_xp
r4_gold
r4_lh
r4_kills
r4_deaths
r4_items
r5_hero
r5_level
r5_xp
r5_gold
r5_lh
r5_kills
r5_deaths
r5_items
d1_hero
d1_level
d1_xp
d1_gold
d1_lh
d1_kills
d1_deaths
d1_items
d2_hero
d2_level
d2_xp
d2_gold
d2_lh
d2_kills
d2_deaths
d2_items
d3_hero
d3_level
d3_xp
d3_gold
d3_lh
d3_kills
d3_deaths
d3_items
d4_hero
d4_level
d4_xp
d4_gold
d4_lh
d4_kills
d4_deaths
d4_items
d5_hero
d5_level
d5_xp
d5_gold
d5_lh
d5_kills
d5_deaths
d5_items
first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_tpscroll_count
radiant_boots_count
radiant_ward_observer_count
radiant_ward_sentry_count
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire

1. (продолжение) Признаки, связанные с итогами матча (помеченные в описании данных как отсутствующие в тестовой выборке) -- это duration, radiant_win, tower_status_radiant, tower_status_dire, barracks_status_radiant, barracks_status_dire. Столбец radiant_win -- это наша целевая переменная, поэтому от нее мы пока не будем избавляться (потом мы разобьем таблицу на матрицу объекты-признаки и целевую переменную).

In [4]:
data_train = data_train.drop(columns=['duration','tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'])
data_train.head(10)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,radiant_win
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,1
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,0
5,1430284186,1,11,5,1961,1461,19,0,1,6,...,-27.0,2.0,-86.0,212.0,4,4,4,0,-43.0,0
8,1430293701,1,8,3,967,1136,7,1,0,8,...,-35.0,6.0,-86.0,182.0,3,6,3,0,10.0,0
9,1430299335,7,35,5,2117,1252,16,0,0,6,...,26.0,119.0,-61.0,NaN,2,3,4,0,-15.0,1
11,1430308974,1,17,5,1527,906,10,0,1,7,...,9.0,NaN,-84.0,226.0,1,3,4,0,26.0,0


2. Проверим выборку на наличие пропусков. Просто вызвав count(), когда у нас 103 столбца, мы ничего не увидим. Придется узнать общее число строк и сравнивать с ним. Выведем количество пропусков в каждом столбце, где они есть. 

In [5]:
data_counter = data_train.count()
data_counter

start_time                  97230
lobby_type                  97230
r1_hero                     97230
r1_level                    97230
r1_xp                       97230
                            ...  
dire_boots_count            97230
dire_ward_observer_count    97230
dire_ward_sentry_count      97230
dire_first_ward_time        95404
radiant_win                 97230
Length: 103, dtype: int64

In [6]:
n_training_examples = len(data_train.index)
print(f"Our DataFrame has {n_training_examples} rows")
for c in data_train.columns:
    if data_counter[c] < n_training_examples:        
        print(f"Column {c} has {n_training_examples - data_counter[c]} missing values")
    #else:
        #print(f"Column {c} has no missing values")

Our DataFrame has 97230 rows
Column first_blood_time has 19553 missing values
Column first_blood_team has 19553 missing values
Column first_blood_player1 has 19553 missing values
Column first_blood_player2 has 43987 missing values
Column radiant_bottle_time has 15691 missing values
Column radiant_courier_time has 692 missing values
Column radiant_flying_courier_time has 27479 missing values
Column radiant_first_ward_time has 1836 missing values
Column dire_bottle_time has 16143 missing values
Column dire_courier_time has 676 missing values
Column dire_flying_courier_time has 26098 missing values
Column dire_first_ward_time has 1826 missing values


2. (продолжение) Видим, что пропущенные значения присутствуют у всех признаков события "первая кровь" (first blood): first_blood_time, first_blood_team, first_blood_player1, first_blood_player2. Как указано в описании данных, эти признаки принимают пропущенное значение, если событие "первая кровь" не успело произойти за первые 5 минут. Заметим, что у признака first_blood_player2 пропущенных значений больше, чем у остальных признаков события "первая кровь" -- поскольку убит может быть не герой (см. описание поля "kills_log" в исходной записи матча). Также пропущенные значения присутствуют у признаков radiant_bottle_time, radiant_courier_time, radiant_flying_courier_time, radiant_first_ward и аналогичных им признаков для команды Dire. Видим, что все это -- времена приобретения командой некоторых предметов (или установки "наблюдателя"). Соответственно, значения будут пропущены, если в первые 5 минут матча приобретения/установки не случилось. 

3. Заменим пропуски на нули с помощью функции fillna().

In [7]:
data_train = data_train.fillna(0)

4. Еще раз заметим, что целевую переменную содержит столбец radiant_win

In [8]:
X_train = data_train.drop(columns = ['radiant_win'])
y_train = data_train['radiant_win']

5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируем генератор разбиений для кросс-валидации по 5 блокам (KFold), перемешав выборку (данные в таблице отсортированы по времени). 

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=241)

5. (продолжение) Оценим качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации.

In [10]:
scores_gradboost = {}
MIN_N_EST = 10
MAX_N_EST = 40
for k in range(MIN_N_EST , MAX_N_EST , 5):
    for lr in [0.5, 0.2, 0.1]:
        print(f'{k} trees, learning rate = {lr}')
        start_time = datetime.datetime.now()
        gbc = GradientBoostingClassifier(n_estimators=k, learning_rate=lr, verbose=False, random_state=241)
        score = round(cross_val_score(gbc, X_train, y_train, cv = kf, scoring='roc_auc').mean(), 3)
        scores_gradboost[(k, lr)] = score
        print(f'Score = {score}')
        print(f'Time elapsed: {datetime.datetime.now() - start_time}')
    

10 trees, learning rate = 0.5
Score = 0.684
Time elapsed: 0:00:35.645761
10 trees, learning rate = 0.2
Score = 0.677
Time elapsed: 0:00:36.795739
10 trees, learning rate = 0.1
Score = 0.664
Time elapsed: 0:00:38.081808
15 trees, learning rate = 0.5
Score = 0.692
Time elapsed: 0:00:59.913918
15 trees, learning rate = 0.2
Score = 0.686
Time elapsed: 0:00:59.846571
15 trees, learning rate = 0.1
Score = 0.676
Time elapsed: 0:01:00.612310
20 trees, learning rate = 0.5
Score = 0.697
Time elapsed: 0:01:19.454491
20 trees, learning rate = 0.2
Score = 0.692
Time elapsed: 0:01:18.608738
20 trees, learning rate = 0.1
Score = 0.683
Time elapsed: 0:01:17.487283
25 trees, learning rate = 0.5
Score = 0.7
Time elapsed: 0:01:37.118081
25 trees, learning rate = 0.2
Score = 0.695
Time elapsed: 0:01:37.478978
25 trees, learning rate = 0.1
Score = 0.687
Time elapsed: 0:01:37.595013
30 trees, learning rate = 0.5
Score = 0.702
Time elapsed: 0:02:01.343362
30 trees, learning rate = 0.2
Score = 0.698
Time elap

In [11]:
scores_gradboost

{(10, 0.5): 0.684,
 (10, 0.2): 0.677,
 (10, 0.1): 0.664,
 (15, 0.5): 0.692,
 (15, 0.2): 0.686,
 (15, 0.1): 0.676,
 (20, 0.5): 0.697,
 (20, 0.2): 0.692,
 (20, 0.1): 0.683,
 (25, 0.5): 0.7,
 (25, 0.2): 0.695,
 (25, 0.1): 0.687,
 (30, 0.5): 0.702,
 (30, 0.2): 0.698,
 (30, 0.1): 0.689,
 (35, 0.5): 0.703,
 (35, 0.2): 0.7,
 (35, 0.1): 0.692}

6. (продолжение) Качество для 20 деревьев: 0.697 при learning rate = 0.5, 0.692 при learning rate = 0.2, 0.683 при learning rate = 0.1. Качество для 30 деревьев: 0.702 при learning rate = 0.5, 0.698 при learning rate = 0.2, 0.689 при learning rate = 0.1. Не похоже, чтобы качество при увеличении числа деревьев сильно возрастало.

### Подход 1. Ответы на вопросы.
В отчете по данному этапу вы должны ответить на следующие вопросы:
1. **Вопрос**: Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)? **Ответ**: Во-первых, пропущенные значения присутствуют у всех признаков события "первая кровь" (first blood): first_blood_time, first_blood_team, first_blood_player1, first_blood_player2. Как указано в описании данных, эти признаки принимают пропущенное значение, если событие "первая кровь" не успело произойти за первые 5 минут. У признака first_blood_player2 пропущенных значений больше, чем у остальных признаков события "первая кровь" -- поскольку убит может быть не герой (см. описание поля "kills_log" в исходной записи матча). Также пропущенные значения присутствуют у признаков radiant_bottle_time, radiant_courier_time, radiant_flying_courier_time, radiant_first_ward и аналогичных им признаков для команды Dire. Видим, что все это -- времена приобретения командой некоторых предметов (или установки "наблюдателя"). Соответственно, значения будут пропущены, если в первые 5 минут матча приобретения/установки не случилось.

2. **Вопрос**: Как называется столбец, содержащий целевую переменную? **Ответ**: Это столбец radiant_win

3. **Вопрос**: Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC. **Ответ**: Время кросс-валидации для градиентного бустинга с 30 деревьями варьируется при разных запусках и составляет примерно 2 минуты (при последнем запуске 02:00.59 при learning rate = 0.5,  02:00.94 при learning rate = 0.2, 01:45.77 при learning rate = 0.1). Качество: 0.702 при learning rate = 0.5, 0.698 при learning rate = 0.2, 0.689 при learning rate = 0.1. 

4. **Вопрос**: Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев? **Ответ**: До сих пор при изменении числа деревьев с 20 до 35 мы не видим существенного повышения качества, так что, скорее всего, не ожидаем его и далее. Если мы все же захотим осуществить такой эксперимент, можно использовать для обучения и кросс-валидации не всю выборку, а некоторое ее случайное подмножество, либо уменьшить глубину деревьев.


### Подход 2: логистическая регрессия. Задания

Здесь буду последовательно описывать процесс выполнения заданий. Ответы на итоговые вопросы еще раз выписаны ниже (под заголовком "Подход 2. Ответы на вопросы).

1. Применим логистическую регрессию с L2-регуляризацией. Проведем кросс-валидацию для параметра регуляризации из списка [1, 5, 10, 20, 50, 100, 500, 1000]. 

In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled

array([[-2.54436416,  1.54068827, -1.24422828, ..., -0.55115386,
         1.84600409, -1.12149424],
       [-2.54045236, -0.92779756, -0.29225805, ...,  0.67817009,
         0.43778816,  0.04394713],
       [-2.53923104,  1.54068827, -0.5686365 , ...,  0.67817009,
         0.43778816,  0.49028637],
       ...,
       [ 1.09874571, -0.57515673,  1.42743012, ...,  0.67817009,
         0.43778816, -0.20401912],
       [ 1.09895204, -0.57515673,  1.48884755, ...,  0.67817009,
         0.43778816, -0.87352799],
       [ 1.1026479 ,  1.54068827, -0.04658831, ..., -0.55115386,
        -0.97042777, -0.79913812]])

In [13]:
scores_logistic = {}
for reg_coeff in [1, 5, 10, 20, 50, 100, 500, 1000]:
    print(f'Regularization coefficient = {reg_coeff}')
    start_time = datetime.datetime.now()
    lrc = LogisticRegression(penalty='l2', C=reg_coeff, random_state=241)
    score = round(cross_val_score(lrc, X_train_scaled, y_train, cv = kf, scoring='roc_auc').mean(), 3)
    scores_logistic[reg_coeff] = score
    print(f'Score = {score}')
    print(f'Time elapsed: {datetime.datetime.now() - start_time}')

Regularization coefficient = 1
Score = 0.716
Time elapsed: 0:00:05.098774
Regularization coefficient = 5
Score = 0.716
Time elapsed: 0:00:05.580320
Regularization coefficient = 10
Score = 0.716
Time elapsed: 0:00:05.579943
Regularization coefficient = 20
Score = 0.716
Time elapsed: 0:00:05.531350
Regularization coefficient = 50
Score = 0.716
Time elapsed: 0:00:05.771267
Regularization coefficient = 100
Score = 0.716
Time elapsed: 0:00:05.664648
Regularization coefficient = 500
Score = 0.716
Time elapsed: 0:00:06.144183
Regularization coefficient = 1000
Score = 0.716
Time elapsed: 0:00:05.038272


In [14]:
scores_logistic

{1: 0.716,
 5: 0.716,
 10: 0.716,
 20: 0.716,
 50: 0.716,
 100: 0.716,
 500: 0.716,
 1000: 0.716}

1. (продолжение) При всех значениях коэффициента регуляризации качество равно 0.716, что незначительно выше качества градиентного бустинга. Видимо, наши данные хорошо приспособлены для моделирования с помощью логистической регрессии -- являются выборкой независимых наблюдений из одного и того же параметрического семейства распределений. Логистическая регрессия по сравнению с градиентным бустингом работает заметно быстрее (примерно за 5 секунд). Видимо, это связано с тем, что вычисления в логистической регрессии хорошо поддаются оптимизации.

2. Уберем категориальные признаки из выборки и проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. 

In [15]:
category_features = [c for c in X_train.columns if c.endswith('hero')]
category_features.append('lobby_type')
X_train_no_cat = X_train.drop(columns = category_features)
X_train_no_cat

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,4,1706,1198,17,0,1,8,2,616,...,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
114403,1450277704,4,1793,1416,17,0,1,5,3,764,...,0,-5.0,0.0,-82.0,0.0,4,3,2,0,-17.0
114404,1450291848,4,1399,540,1,0,0,5,4,1448,...,2,-32.0,249.0,-70.0,0.0,1,1,3,1,-15.0


In [16]:
X_train_no_cat_scaled = scaler.fit_transform(X_train_no_cat)
X_train_no_cat_scaled

array([[-2.54436416,  1.40080818,  1.52597175, ..., -0.55115386,
         1.84600409, -1.12149424],
       [-2.54045236,  0.50131354, -0.08013929, ...,  0.67817009,
         0.43778816,  0.04394713],
       [-2.53923104,  0.50131354,  0.1510701 , ...,  0.67817009,
         0.43778816,  0.49028637],
       ...,
       [ 1.09874571,  0.50131354,  0.29226667, ...,  0.67817009,
         0.43778816, -0.20401912],
       [ 1.09895204, -0.39818111, -0.17368203, ...,  0.67817009,
         0.43778816, -0.87352799],
       [ 1.1026479 , -0.39818111, -0.31840851, ..., -0.55115386,
        -0.97042777, -0.79913812]])

In [17]:
scores_logistic_no_cat = {}
for reg_coeff in [1, 5, 10, 20, 50, 100, 500, 1000]:
    print(f'Regularization coefficient = {reg_coeff}')
    start_time = datetime.datetime.now()
    lrc = LogisticRegression(penalty='l2', C=reg_coeff, random_state=241)
    score = round(cross_val_score(lrc, X_train_no_cat_scaled, y_train, cv = kf, scoring='roc_auc').mean(), 3)
    scores_logistic_no_cat[reg_coeff] = score
    print(f'Score = {score}')
    print(f'Time elapsed: {datetime.datetime.now() - start_time}')


Regularization coefficient = 1
Score = 0.716
Time elapsed: 0:00:05.013648
Regularization coefficient = 5
Score = 0.716
Time elapsed: 0:00:05.409200
Regularization coefficient = 10
Score = 0.716
Time elapsed: 0:00:05.048081
Regularization coefficient = 20
Score = 0.716
Time elapsed: 0:00:04.770871
Regularization coefficient = 50
Score = 0.716
Time elapsed: 0:00:04.875883
Regularization coefficient = 100
Score = 0.716
Time elapsed: 0:00:05.085290
Regularization coefficient = 500
Score = 0.716
Time elapsed: 0:00:05.794346
Regularization coefficient = 1000
Score = 0.716
Time elapsed: 0:00:05.525259


In [18]:
scores_logistic_no_cat

{1: 0.716,
 5: 0.716,
 10: 0.716,
 20: 0.716,
 50: 0.716,
 100: 0.716,
 500: 0.716,
 1000: 0.716}

2. (продолжение) Качество не изменилось, осталось 0.716. Это можно объяснить тем, что категориальные признаки были нам не очень важны -- действительно, вряд ли на результат команды влияет то, кому именно из игроков команды принадлежит конкретный герой.

3. Выясним из данных, сколько различных идентификаторов героев существует в игре.

In [19]:
N = 0
for p in range(5):
    r_unique = X_train['r%d_hero'%(p + 1)].unique()
    d_unique = X_train['d%d_hero'%(p + 1)].unique()
    N = max(N, max(r_unique), max(d_unique))
N

112

4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Сформируем (с помощью готового кода от авторов курса) N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire.

In [20]:
X_train_pick = np.zeros((X_train.shape[0], N))
for i, match_id in enumerate(X_train.index):
    for p in range(5):
        X_train_pick[i, X_train.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_train_pick[i, X_train.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_train_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

4. (продолжение) Добавим полученные признаки к числовым, которые мы использовали во втором пункте данного этапа. 

In [21]:
X_train_with_pick = np.hstack((X_train_no_cat, X_train_pick))
X_train_with_pick_scaled = scaler.fit_transform(X_train_with_pick)
X_train_with_pick_scaled

array([[-2.54436416e+00,  1.40080818e+00,  1.52597175e+00, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [-2.54045236e+00,  5.01313538e-01, -8.01392943e-02, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [-2.53923104e+00,  5.01313538e-01,  1.51070097e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       ...,
       [ 1.09874571e+00,  5.01313538e-01,  2.92266672e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  1.64609695e+00],
       [ 1.09895204e+00, -3.98181106e-01, -1.73682025e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [ 1.10264790e+00, -3.98181106e-01, -3.18408515e-01, ...,
        -4.83865990e-03,  0.00000000e+00, -1.64440483e+00]])

5. Проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. 

In [22]:
scores_logistic_with_pick = {}
for reg_coeff in [1, 5, 10, 20, 50, 100, 500, 1000]:
    print(f'Regularization coefficient = {reg_coeff}')
    start_time = datetime.datetime.now()
    lrc = LogisticRegression(penalty='l2', C=reg_coeff, random_state=241)
    score = round(cross_val_score(lrc, X_train_with_pick_scaled, y_train, cv = kf, scoring='roc_auc').mean(), 3)
    scores_logistic_with_pick[reg_coeff] = score
    print(f'Score = {score}')
    print(f'Time elapsed: {datetime.datetime.now() - start_time}')

Regularization coefficient = 1
Score = 0.752
Time elapsed: 0:00:10.758676
Regularization coefficient = 5
Score = 0.752
Time elapsed: 0:00:10.643397
Regularization coefficient = 10
Score = 0.752
Time elapsed: 0:00:11.182163
Regularization coefficient = 20
Score = 0.752
Time elapsed: 0:00:11.390642
Regularization coefficient = 50
Score = 0.752
Time elapsed: 0:00:10.887132
Regularization coefficient = 100
Score = 0.752
Time elapsed: 0:00:10.909727
Regularization coefficient = 500
Score = 0.752
Time elapsed: 0:00:10.326352
Regularization coefficient = 1000
Score = 0.752
Time elapsed: 0:00:10.143880


5. (продолжение) Качество при всех значениях коэффициента регуляризации стало равным 0.752.

6. Построим предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей. Это -- логистическая регрессия, обученная с учетом информации о героях. Коэффициент регуляризации берем равным 10, но можно было бы взять любой другой из протестированных (качество было одинаковым).

In [23]:
reg_coeff = 10
best_clf = LogisticRegression(penalty='l2', C=reg_coeff, random_state=241)
best_clf.fit(X_train_with_pick_scaled, y_train)

LogisticRegression(C=10, random_state=241)

In [24]:
X_test = pd.read_csv('./features_test.csv', index_col='match_id')
X_test = X_test.fillna(0)

In [25]:
X_test_no_cat = X_test.drop(columns = category_features)
X_test_no_cat

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,4,1103,1089,8,0,1,9,3,1183,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,2,556,570,1,0,0,9,4,1194,...,2,-29.0,168.0,-54.0,0.0,3,2,2,1,16.0
10,1430301774,2,751,808,1,0,0,13,2,421,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,3,708,903,1,1,1,11,2,672,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,4,1259,661,4,0,0,9,5,1703,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114369,1450212780,5,2054,1941,27,0,1,8,4,1460,...,1,8.0,253.0,-87.0,0.0,4,3,2,1,-33.0
114377,1450222875,3,748,605,1,0,0,12,3,1130,...,1,-1.0,133.0,-85.0,184.0,2,3,4,1,-18.0
114378,1450223593,2,575,499,0,0,0,8,3,697,...,0,20.0,133.0,-88.0,239.0,4,4,4,0,-36.0


In [26]:
X_test_pick = np.zeros((X_test.shape[0], N))
for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_test_pick[i, X_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_test_pick[i, X_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_test_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  0., ...,  0.,  0., -1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [27]:
X_test_with_pick = np.hstack((X_test_no_cat, X_test_pick))
X_test_with_pick_scaled = scaler.transform(X_test_with_pick)
X_test_with_pick_scaled

array([[-2.52819967e+00,  5.01313538e-01, -2.30160655e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [-2.52721443e+00, -1.29767575e+00, -1.19559224e+00, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [-2.52568833e+00, -1.29767575e+00, -8.51425585e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  1.64609695e+00],
       ...,
       [ 1.08637028e+00, -1.29767575e+00, -1.16205805e+00, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [ 1.09021010e+00,  5.01313538e-01,  1.07767262e+00, ...,
        -4.83865990e-03,  0.00000000e+00, -1.64440483e+00],
       [ 1.09214252e+00,  5.01313538e-01, -3.24854502e-02, ...,
        -4.83865990e-03,  0.00000000e+00,  1.64609695e+00]])

In [28]:
best_clf.classes_

array([0, 1])

In [29]:
prob_pred = best_clf.predict_proba(X_test_with_pick_scaled)
prob_pred

array([[0.17493703, 0.82506297],
       [0.2454172 , 0.7545828 ],
       [0.81273401, 0.18726599],
       ...,
       [0.76477953, 0.23522047],
       [0.37071432, 0.62928568],
       [0.57559346, 0.42440654]])

Получили массив вероятностей класса 0 (выиграли Dire) и класса 1 (выиграли Radiant) -- в том же порядке, что в best_clf.classes_

In [30]:
y_pred = prob_pred[:, 1]
y_pred

array([0.82506297, 0.7545828 , 0.18726599, ..., 0.23522047, 0.62928568,
       0.42440654])

6. (продолжение) Предсказанные вероятности находятся на отрезке [0, 1] и не совпадают между собой (т.е. что модель не получилась константной).

In [31]:
min(y_pred), max(y_pred)

(0.008460355689552104, 0.9964380981573405)

### Подход 2. Ответы на вопросы.
1. **Вопрос**: Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом? **Ответ**:  При всех значениях коэффициента регуляризации качество равно 0.716, что незначительно выше качества градиентного бустинга. Видимо, наши данные хорошо приспособлены для моделирования с помощью логистической регрессии -- являются выборкой независимых наблюдений из одного и того же параметрического семейства распределений. Логистическая регрессия по сравнению с градиентным бустингом работает заметно быстрее (примерно за 5 секунд). Видимо, это связано с тем, что вычисления в логистической регрессии хорошо поддаются оптимизации.
2. **Вопрос**: Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение? **Ответ**: Качество не изменилось, осталось 0.716. Это можно объяснить тем, что категориальные признаки были нам не очень важны -- действительно, вряд ли на результат команды влияет то, кому именно из игроков команды принадлежит конкретный герой.
3. **Вопрос**: Сколько различных идентификаторов героев существует в данной игре? **Ответ**: 112
4. **Вопрос**:Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить? **Ответ**: Качество улучшилось до 0.752. По-видимому, состав команды (наличие или отсутствие  в ней определенных героев) важен для исхода игры.
5. **Вопрос**: Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов? **Ответ**: минимальное значение 0.008460355689552104, максимальное 0.9964380981573405
